![](https://pbs.twimg.com/profile_images/1023997768577179648/WeOL7-oq_400x400.jpg)
As the leading healthcare organization for informatics in medical imaging, the Society for Imaging Informatics in Medicine (SIIM)'s mission is to advance medical imaging informatics through education, research, and innovation in a multi-disciplinary community. SIIM is joined by the International Skin Imaging Collaboration (ISIC), an international effort to improve melanoma diagnosis. The ISIC Archive contains the largest publicly available collection of quality-controlled dermoscopic images of skin lesions.

In this competition, you’ll identify melanoma in images of skin lesions. In particular, you’ll use images within the same patient and determine which are likely to represent a melanoma. Using patient-level contextual information may help the development of image analysis tools, which could better support clinical dermatologists.

Thanks to my team for their constant efforts made to this competition, that paid off with a silver medal.

1. Rohit Agarwal (https://www.kaggle.com/rohitagarwal)
2. Vladmir (https://www.kaggle.com/vladimirsydor)
3. Ashish Gupta (https://www.kaggle.com/roydatascience)
4. Mukharbek Organokov (https://www.kaggle.com/muhakabartay)

In this kernel, I used top 14 models, our team built - using Pseudo Labelling approach.

In [ ]:
import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from subprocess import check_output
print(check_output(["ls", "../input/"]).decode("utf8"))

In [ ]:
from scipy.stats import rankdata
import glob
LABELS = ["target"]
all_files = glob.glob("../input/pseudolabelmodels/*.csv")

In [ ]:
outs = [pd.read_csv(f, index_col=0) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "m" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
rank = np.tril(concat_sub.iloc[:,1:].corr().values,-1)
m = (rank>0).sum()
m_gmean, s = 0, 0
for n in range(min(rank.shape[0],m)):
    mx = np.unravel_index(rank.argmin(), rank.shape)
    w = (m-n)/(m+n/10)
    #print(w)
    m_gmean += w*(np.log(concat_sub.iloc[:,mx[0]+1])+np.log(concat_sub.iloc[:,mx[1]+1]))/2
    s += w
    rank[mx] = 1
m_gmean = np.exp(m_gmean/s).clip(0.0,1.0)

In [ ]:
predict_list = []
predict_list.append(pd.read_csv('../input/pseudolabelmodels/submission-9350-pseudo-rohit.csv')[LABELS].values)
predict_list.append(pd.read_csv('../input/pseudolabelmodels/submission-9351-pseudo-rohit.csv')[LABELS].values)
predict_list.append(pd.read_csv('../input/pseudolabelmodels/submission-9373-pseudo-vova.csv')[LABELS].values)
predict_list.append(pd.read_csv('../input/pseudolabelmodels/submission-9376-pseudo-rohit.csv')[LABELS].values)
predict_list.append(pd.read_csv('../input/pseudolabelmodels/submission-9379-pseudo-rohit.csv')[LABELS].values)
predict_list.append(pd.read_csv('../input/pseudolabelmodels/submission-9432-pseudo-vova.csv')[LABELS].values)
predict_list.append(pd.read_csv('../input/pseudolabelmodels/submission-9474-pseudo-rohit.csv')[LABELS].values)


predict_list_2 = []
predict_list_2.append(pd.read_csv('../input/pseudolabelmodels/submission-9477-pseudo-ashish.csv')[LABELS].values)
predict_list_2.append(pd.read_csv('../input/pseudolabelmodels/submission-9491-pseudo-rohit.csv')[LABELS].values)
predict_list_2.append(pd.read_csv('../input/pseudolabelmodels/submission-94981-pseudo-ashish.csv')[LABELS].values)
predict_list_2.append(pd.read_csv('../input/pseudolabelmodels/submission-94982-pseudo-ashish.csv')[LABELS].values)
predict_list_2.append(pd.read_csv('../input/pseudolabelmodels/submission-9508-pseudo-rohit.csv')[LABELS].values)
predict_list_2.append(pd.read_csv('../input/pseudolabelmodels/submission-9513-pseudo-ashish.csv')[LABELS].values)
predict_list_2.append(pd.read_csv('../input/pseudolabelmodels/submission-9547-pseudo-rohit.csv')[LABELS].values)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
print("Rank averaging on ", len(predict_list), " files")
predictions = np.zeros_like(predict_list[0])
for predict in predict_list:
    for i in range(1):
        predictions[:, i] = np.add(predictions[:, i], rankdata(predict[:, i])/predictions.shape[0])  

predictions = predictions /len(predict_list)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
print("Rank averaging on ", len(predict_list_2), " files")
predictions_2 = np.zeros_like(predict_list_2[0])
for predict_2 in predict_list_2:
    for i in range(1):
        predictions_2[:, i] = np.add(predictions_2[:, i], rankdata(predict_2[:, i])/predictions_2.shape[0])  

predictions_2 = predictions_2 /len(predict_list_2)

In [ ]:
submission_1 = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')
submission_1[LABELS] = predictions

submission_2 = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')
submission_2[LABELS] = predictions_2

submission = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')
submission[LABELS] = submission_2[LABELS]*.55 + submission_1[LABELS]*.45


submission.to_csv('submission.csv', index=False)